**Nombre:** Sebastián Numpaque

**Documento:** 1002396960

---

#**Importe de Librerias**

In [1]:
# @title

# Array Management
import numpy as np

# Numerical Calculus
import astropy.units as u
import astropy.constants as const

# Plotting
import plotly.graph_objects as go

# Numerical Integration
from scipy.integrate import quad

# Distance Calculus
from astropy.coordinates import SkyCoord

# Planck´s Intensity Distribution
from astropy.modeling.physical_models import BlackBody

# Stars Distribution
from astropy.coordinates import uniform_spherical_random_volume



---

#**Exercise a)**

Dentro de esta celda definimos las siguientes variables:

- `N_Ostars`: Número de Estrellas Tipo O

- `L_Ostars`: Luminosidad de Estrellas Tipo O

- `r_gal`: Radio de la Galaxia

Aprovechando la clase `SkyCoord` y la función `uniform_spherical_random_volume` del módulo `astropy.coordinates` generamos una distribución uniforme de estrellas sobre un plano circular (z = 0) con representación cartesiana. Este conjunto de coordenadas se almacenan en la variable `rs_Ostars`.

In [2]:
# @title

# Number of O-Type Stars
N_Ostars = 5000

# O-Type Stars Luminosity (erg/s)
L_Ostars = 10e40*u.erg/u.s

# Galaxy Radius (kpc)
r_gal = 36*u.kpc

# O-Type Stars Distribution
rs_Ostars = SkyCoord(uniform_spherical_random_volume(size = N_Ostars,
                                                    max_radius = r_gal
                                                    ).to_cartesian(),
                    representation_type = 'cartesian')

rs_Ostars.z[()] = np.zeros_like(rs_Ostars.z)

In [3]:
# @markdown Plotting Stars Distribution
go.Figure(data = [go.Scatter(x = rs_Ostars.x,
                             y = rs_Ostars.y,
                             showlegend = False,
                             marker_size = 3,
                             marker_color = '#00CC96',
                             mode = 'markers')],

          layout = go.Layout(title_text = 'O-Type Stars Distribution',
                             height = 630, width = 740,
                             title_x = 0.5, xaxis_title = 'X [kpc]',
                             yaxis_title = 'Y [kpc]'))

#**Exercise b)**

Mediante el atributo `self.separation_3d` de la clase `SkyCoord` calculamos la distancia desde cada una de las estrellas contenidas en `rs_Ostars` hacia una region dentro de la galaxia ($0 < r < r_{gal}$) en intervalos de **1** $kpc$. Estas distancias se almacenan en la variable `ds_Ostars`

Siguiendo la expresión del Flujo a partir de la Luminosidad, consideramos la contribución de cada una de las estrellas sobre el Flujo en el Medio Interestelar, donde $d_i$ corresponde a la distancia respectiva entre cada estrella de la distribución y la región de la galaxia, contenida en `ds_Ostars`.

$$F_{ISM}=\sum_{i=1}^{N_{O*}}F_i=\sum_{i=1}^{N_{O*}}\frac{L_{O*}}{4\pi d_i^2}$$

Para evitar las anomalías de cercanía con las estrellas, para cada una de las regiones generamos 50 mediciones de distancia. De esta forma, se calcularon 50 mediciones en el Flujo del Medio Interestelar por región. Entre dichas mediciones tomamos el valor mínimo y sus valores se almacenan en la variable `F_ISM`.

In [4]:
# @title

# Distance Calculus
ds_Ostars = np.array([[rs_Ostars.separation_3d(SkyCoord(ra, 0,
                                                        distance = r,
                                                        unit = (u.rad,
                                                                u.rad,
                                                                u.kpc)))
                      for ra in np.linspace(0, 2*np.pi, 50)]
                      for r in range(37)])*u.kpc

# Interstellar Flux Calculus (erg/cm^2 s)
F_ISM = np.min(np.sum(L_Ostars/(4*np.pi*ds_Ostars**2), axis = 2),
               axis = 1).to(u.erg/(u.s*u.cm**2))

In [5]:
#@markdown Plotting Interstellar Medium Flux
go.Figure(data = [go.Scatter(x = np.arange(37),
                             y = F_ISM)],

          layout = go.Layout(title_text = 'ISM Flux',
                             height = 600, width = 900,
                             xaxis_title = 'R [kpc]',
                             yaxis_title = 'Flux [erg/s cm^2]',
                             title_x = 0.5))



---

#**Exercise c)**

El fenómeno de Extinción ocasiona una disminución sobre el Flujo real de las estrellas, es decir, una percepción en el aumento sobre las distancias a las que se encuentran.

$$d_{apparent} = d_{real}10^\frac{a_\lambda d_{real}}{5}$$

Calculamos nuevamente el Flujo, siguiendo el procedimiento del literal anterior y considerando las nuevas distancias, donde $d_{real}$ hace referencia a las distancias contenidas en `ds_Ostars`.

In [6]:
# @title

# Extinction Coefficient (mag/kpc)
a_ext = 1.5/u.kpc

# Aparent Distance
ds_apparent = ds_Ostars*(10**(a_ext*ds_Ostars/5))

# Interstellas Flux with Extinction (erg/cm^2 s)
F_ISM_ext = np.min(np.sum(L_Ostars/(4*np.pi*ds_apparent**2),
                  axis = 2), axis = 1).to(u.erg/(u.s*u.cm**2))

In [7]:
#@markdown Plotting InterStellar Medium Flux (Extinction)
go.Figure(data = [go.Scatter(x = np.arange(37),
                             y = F_ISM_ext)],

          layout = go.Layout(title_text = 'ISM Flux' +\
                                          '<br>(aλ = 1.5 mag/kpc)',
                             height = 600, width = 900,
                             xaxis_title = 'R [kpc]',
                             yaxis_title = 'Flux [erg/s cm^2]',
                             title_x = 0.5))



---

#**Exercise d)**

Definimos las siguientes variables

- `T_Ostars`: Temperatura de las Estrellas Tipo O

- `lamda_Ostars`: Longitud de Onda del Pico de Emisión siguiendo la Ley de Desplazamiento de Wien.

  $$\lambda = \frac{b}{T}$$

- `E_photon`: Energía del Fotón para dicha Longitud de Onda

  $$E_\gamma=\frac{hc}{\lambda}$$

Asumiendo que el Flujo Bolométrico del Medio Interestelar es producido únicamente por los Fotones del Pico de Emisión, el Flujo de Fotones equivale simplemente al cociente entre los valores de Flujo Bolométrico y la Energía de los Fotones.

Bajo esta suposición, y considerando los efectos de Extinción, obtenemos valores del orden de $10^{-8}\,\gamma/s\,cm^2$ siendo buena aproximación al valor reportado en la literatura [1]. Esto indicaría que las principales fuentes de radiación en el Medio Interestelar están asociadas a Estrellas de Tipo O a pesar de que representan un bajo porcentaje (~1e-7 %) entre las poblaciones estelares de una Galaxia.
- [1] https://www.science.org/doi/10.1126/sciadv.aao2538




In [8]:
# @title

# O-Type Stars Temperature (K)
T_Ostars = 4e4*u.K

# Wien O-Type Stars Wavelength
lamda_Ostars = const.b_wien/T_Ostars

# Photon Energy
E_photon = const.h*const.c/lamda_Ostars

# Photon Flux
F_photon = F_ISM_ext/E_photon.to(u.erg)

In [9]:
#@markdown Plotting Photon Flux
go.Figure(data = [go.Scatter(x = np.arange(37),
                             y = F_photon)],

          layout = go.Layout(title_text = 'ISM Photon Flux' +\
                                          '<br>(aλ = 1.5 mag/kpc)',
                             height = 600, width = 900,
                             xaxis_title = 'R [kpc]',
                             yaxis_title = 'Flux γ [s^-1 cm^-2]',
                             yaxis_exponentformat = 'e',
                             title_x = 0.5)).show()



---

#**Exercise e)**

Bajo todas las consideraciones propuestas en el planteamiento del problema realizamos el siguiente desarrollo para la Temperatura de Equilibrio del Polvo:


$$\begin{align*}
F_{ISM}&\approx Q_\lambda\sigma\,T_g^4\\
&=\left(\frac{2\pi a}{\lambda_g}\right)^\beta \sigma\,T_g^{4}\\
&=\left(\frac{2\pi a}{b}\right)^\beta \sigma\,T_g^{4+\beta}
\end{align*}$$

$$T_g \approx \left[\frac{F_{ISM}}{\sigma}\left(\frac{b}{2\pi a}\right)^\beta\right]^{\frac{1}{4+\beta}}$$

Calculamos $T_g$ para cada regíón ($0< r< r_{gal}$) de la Galaxia obteniendo un valor medio ~16 K cercano al reportado en la literatura [2].

- [2] https://ned.ipac.caltech.edu/level5/Sept14/Conroy/Conroy6.html



In [10]:
# @title
#Parameters
a_dust, beta = (1e-7*u.m, 1.45)

#Dust Equilibrium Temperature
T_dust = (((F_ISM_ext/const.sigma_sb).decompose() *\
          ((const.b_wien/(2*np.pi*a_dust))**beta))**(1/(4 + beta))).value*u.K

In [11]:
#@markdown Plotting Dust Equilibrium Temperature
go.Figure(data = [go.Scatter(x = np.arange(37),
                             y = T_dust)],

          layout = go.Layout(title_text = 'Dust Equilibrium Temperature',
                             height = 600, width = 900,
                             xaxis_title = 'R [kpc]',
                             yaxis_title = 'Tg [K]',
                             title_x = 0.5))



---


#**Exercise f)**

Definimos y calculamos las siguientes variables.

- `n_dust`: Densidad de Número del Polvo

- `sigma_geom`: Sección Geométrica Efectiva

  $$\sigma_g = \pi a^2$$

- `lamda_dust`: Longitud de Onda del Pico de Emisión del Polvo bajo la Ley de Desplazamiento de Wien

  $$\lambda_g = \frac{b}{T_g}$$

- `Q_lamda`: Coeficiente de Extinción de Mie

  $$Q_\lambda = \left(\frac{2\pi a}{\lambda_g}\right)^\beta$$

- `a_ext_dust`: Coeficiente de Extinción del Polvo

  $$a_{\lambda_g}=1.086\,Q_\lambda\,\sigma_g\,n_d$$

Con este calculo obtenemos un valor medio de $a_{\lambda_g}\sim 10^{-5}$ bastantes órdenes por debajo del valor típico $a_\lambda = 1.5\,\frac{mag}{kpc}$.

Bajo la suposición del literal anterior en la que los granos de polvo reemiten toda la radiación absorbida, no se presenta extinción efectiva del Flujo. Del mismo modo, entendiendo que la extinción es proporcional a la frecuencia de la radiación, para longitudes de onda dentro del Infrarrojo se presenta una extinción casi nula.

$$A_\lambda \propto \frac{1}{\lambda}\,;\quad \lambda=\lambda_g\to A_\lambda \approx 0$$

$$A_\lambda=-2.5\log_{10}\left(\frac{F_{ISM}}{Q_\lambda\,F_{SB}}\right)\approx 0\,;\qquad F_{ISM}\approx Q_\lambda\,F_{SB}$$

In [22]:
# @title

# Dust Number Density
n_dust = 2e-13/u.cm**3

# Geometric Cross Section
sigma_geom = np.pi*a_dust**2

# Dust Wien Wavelenght
lamda_dust = const.b_wien/T_dust

# Mie's Extinction Coefficient
Q_lamda = (2*np.pi*a_dust/lamda_dust)**beta

# Extinction Coefficient
a_ext_dust = (1.086*Q_lamda*sigma_geom*n_dust).to(1/u.kpc)

print(f'El Coeficiente Medio de Extinción es:\naλ = {np.mean(a_ext_dust)*u.mag}')

El Coeficiente Medio de Extinción es:
aλ = 5.8891815372250094e-05 mag / kpc




---

#**Exercise g)**

Calculamos el valor de intensidad $I_\nu$ pra el centro de la galaxia bajo la siguiente expresión.

$$I_\nu = S_\nu(1-e^{-\tau_\nu})$$

Utilizando las temperaturas en el centro de la galaxia y su pico de frecuencia asociado, tomamos la función fuente según la distribución de Planck. Para ello empleamos el módelo `astropy.modeling.physical_models.BlackBody`

$$S_\nu=B_\nu(T_{g_0})$$

Finalmente, la profundidad optica es calculada asumiendo un medio homogeneo.

$$\tau_\nu =\int_0^{r_{gal}} \frac{a_\nu(T_{g_0})}{1.086}\,ds = \frac{a_\nu(T_{g_0})\,r_{gal}}{1.086}$$

Siguiendo este desarrollo, obtenemos un valor de $I_\nu\sim 10^{-14}$ erg / s cm² Hz sr confirmando valores de las observaciones [3]

- [3] https://indico.cern.ch/event/506272/contributions/2167227/attachments/1275357/1891894/Ganga.pdf

In [85]:
# @title

# Optical Depth
tau = a_ext_dust[0]*r_gal/1.086

# Source Function
Sv = BlackBody(T_dust[0])

# Radiative Transfer Intensity
Iv_center =  Sv(Sv.nu_max)*(1 - np.exp(-tau))

print(f'La Intensidad en el Centro de la Galaxia es:\nIv = {Iv_center}')

La Intensidad en el Centro de la Galaxia es:
Iv = 1.0738901692037032e-14 erg / (Hz s sr cm2)




---

#**Exercise h)**

Definimos la función de integración `f_int` siguiendo la expresión para el calculo del coeficiente de absorción $k_\lambda$ bajo una distribución sobre las dimensiones del polvo en el Medio Interestelar

$$k_\lambda=\int Q_\lambda\sigma_{geom}\frac{dn_a}{da}\,da = \int \pi\,C\,\left(\frac{2\pi a}{\lambda}\right)^\beta a^{-1.5}\,da$$

Tomando todas las consideraciones sobre el Coeficiente de Extinción de Mie, respetando las condiciones sobre el valor de $\beta$

$$2\pi a< \lambda_{O*}\to\beta=1.45\,; \qquad \lambda_{O*}< 2\pi a\to\beta = 0$$

Realizamos la integración sobre el rango de dimensiones del polvo mediante el módulo `scipy.integrate.quad`.

$$k_{\lambda_{O*}} = \int_0^{\frac{\lambda_{O*}}{2\pi}} \pi\,C\,\left(\frac{2\pi a}{\lambda}\right)^\beta a^{-1.5}\,da + \int_{\frac{\lambda_{O*}}{2\pi}}^{0.09\,\mu m} \pi\,C\,a^{-1.5}\,da$$

Finalmente calculamos el Coeficiente de Extinción, obteniendo un valor $a_{\lambda_{O*}}=1.46$ poco discrepante del valor referido $a_\lambda = 1.5$

In [14]:
# @title

# Integration Function
f_int = lambda a, lamda, beta: C*np.pi*((2*np.pi*a/lamda)**beta)*(a**-1.5)

# Dust Distribution Constant
C = 6.4e-26*u.cm.to(u.nm)**-0.5

# Integration in nm
a_ext = 1.086/u.nm.to(u.kpc) *\
        (quad(f_int, 0,
              lamda_Ostars.to(u.nm).value/(2*np.pi),
              args = (lamda_Ostars.to(u.nm).value, beta))[0] +

        quad(f_int, lamda_Ostars.to(u.nm).value/(2*np.pi), 90,
            args = (lamda_Ostars.to(u.nm).value, 0))[0])

print(f'El Coeficiente de Extinción es:\naλ = {a_ext*u.mag}')

El Coeficiente de Extinción es:
aλ = 1.46625734641834 mag


Siguiendo el anterior desarrollo, calculamos el Coeficiente de Extinción ($a_{\lambda_B}$ y $a_{\lambda_V}$) para las Longitudes de Onda asociadas a los filtros fotométricos B y V (440 nm y 550 nm).

Obtenidas estas cantidades, calculamos el Radio Fiducial ($R_V\approx 3$), verificando la regla empirica respecto a esta cantidad ($R_V\approx 3.1$)

$$R_V=\frac{a_{\lambda_V}}{a_{\lambda_B}-a_{\lambda_V}}$$

**Nota:** Al emplear la integración númerica a través de `scipy.integrate.quad` puede que se encuentren variaciones númericas respecto a soluciones analíticas de la integral para $k_\lambda$; aún así se logra muy buena aproximación.

In [15]:
# @title

# Integration for B Wavelenght in nm
a_ext_B = 1.086/u.nm.to(u.kpc) *\
          (quad(f_int, 0, 440/(2*np.pi),
               args = (440, beta))[0] +

          quad(f_int, 440/(2*np.pi), 90,
               args = (440, 0))[0])

# Integration for V Wavelenght in nm
a_ext_V = 1.086/u.nm.to(u.kpc) *\
          (quad(f_int, 0, 550/(2*np.pi),
                args = (550, beta))[0] +

          quad(f_int, 550/(2*np.pi), 90,
               args = (550, 0))[0])

# Fiducial Ratio
Rv = a_ext_V/(a_ext_B - a_ext_V)

print(f'El valor del Radio Fiducial es:\nRv = {Rv}')

El valor del Radio Fiducial es:
Rv = 2.9979001330588244
